In [45]:
import torch
import random

from sent_order.conll import Corpus, Document

from torch.nn import functional as F
from itertools import islice
from boltons.iterutils import pairwise, chunked
from tqdm import tqdm_notebook
from textblob import TextBlob

from IPython.core.display import display, HTML

In [5]:
model = torch.load(
    '../../data/coref.9.bin',
    map_location={'cpu': 'cpu'},
)

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'sent_order.models.coref.DocEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [6]:
model.eval()

Coref(
  (encode_doc): DocEncoder(
    (embeddings): WordEmbedding(39414, 300)
    (lstm): LSTM(300, 200, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (score_spans): SpanScorer(
    (attention): Scorer(
      (score): Sequential(
        (0): Linear(in_features=400, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
    (width_embeddings): DistanceEmbedding(9, 20)
    (sm): Scorer(
      (score): Sequential(
        (0): Linear(in_features=1120, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
  )
  (score_pairs): PairScorer(
    (dist_embeddings): DistanceEmbedding(9, 20)
    (sa): Scorer(
      (score): Sequential(
        (0): Linear(

In [43]:
c = Corpus.from_combined_file('../../data/development.conll')

In [73]:
correct, total = 0, 0
for doc in tqdm_notebook(c.documents[:500]):
    for s1, s2 in pairwise(doc.sents()):

        try:

            tokens1 = [t.text for t in s1]
            tokens2 = [t.text for t in s2]

            d1 = Document.from_text(' '.join(tokens1 + tokens2))
            d2 = Document.from_text(' '.join(tokens2 + tokens1))
            
            spans1 = model(d1)
            spans2 = model(d2)

            score1 = sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans1]).item()
            score2 = sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans2]).item()
        
            if score1 > score2:
                correct += 1
            
            total += 1

        except Exception as e:
            print(e)

HBox(children=(IntProgress(value=0, max=343), HTML(value='')))

No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate pairs after pruning.
No candidate p

In [74]:
correct / total

0.5202798731824642

In [75]:
total

9147

In [66]:
doc = TextBlob("""
Randall L. Stephenson, AT&T’s chief executive, said on Friday that the company had made a “big mistake” by hiring President Trump’s personal lawyer, Michael D. Cohen, to advise on the telecommunications giant’s deal to buy Time Warner.

“Our company has been in the headlines for all the wrong reasons these last few days and our reputation has been damaged,” Mr. Stephenson wrote in a memo to employees. “There is no other way to say it — AT&T hiring Michael Cohen as a political consultant was a big mistake.”

Mr. Stephenson’s note followed the revelation that the company had paid Mr. Cohen $600,000 to advise on the $85.4 billion merger with Time Warner and other regulatory matters. Mr. Stephenson also said in the memo that the company’s head of lobbying and external affairs, Bob Quinn, 57, would be retiring.

Federal prosecutors are investigating Mr. Cohen’s business dealings, including a $130,000 payment he made to the pornographic film actress Stephanie Clifford, known professionally as Stormy Daniels, as part of a deal to buy her silence about an affair she says she had with Mr. Trump. The president has denied Ms. Clifford’s claims.

The payment to Ms. Clifford was the first known activity involving Essential Consultants, a so-called shell company incorporated in Delaware by Mr. Cohen. It was through Essential Consultants that AT&T retained Mr. Cohen.

AT&T was one of multiple companies Mr. Cohen approached after the election. The Swiss drugmaker Novartis paid Essential Consultants $1.2 million for a yearlong contract to provide insights on the new administration’s approach to health care policy.

Novartis said its former chief executive, Joe Jimenez, hired Essential Consultants. Like Mr. Stephenson, Norvartis’ current chief executive, Vasant Narasimhan, also distanced himself from Mr. Cohen, saying this week that he had no role in the decision to hire Mr. Cohen.

Novartis said it discovered soon after signing the contract that Mr. Cohen could not provide the services he had promised and allowed the contract to expire.

For AT&T, the disclosure of its ties to Mr. Cohen comes at a critical moment. The company is defending its merger with Time Warner in federal court against the Justice Department’s efforts to block the deal.

Mr. Stephenson insisted in his memo that “everything we did was done according to the law and entirely legitimate” and that Mr. Cohen did not do any lobbying on behalf of AT&T. Nonetheless, Mr. Stephenson added, retaining Mr. Cohen “was a serious misjudgment.”

“In this instance,” he continued, “our Washington D.C. team’s vetting process clearly failed, and I take responsibility for that.”

Time Warner was not aware of AT&T’s contract with Mr. Cohen, according to a person familiar with the company’s thinking. Within Time Warner this week, officials were surprised to learn about the contract with Essential Consultants.

AT&T has said that Mr. Cohen approached employees in the company’s Washington office shortly after the election. The company signed a contract with him three days after Mr. Trump took the oath of office. He promised to provide advice on who the “key players” would be in Mr. Trump’s administration and “their priorities, and how they think,” AT&T said in a fact sheet released on Friday.

AT&T noted in the fact sheet that Mr. Cohen was among “several consultants” the company hired as Mr. Trump was assuming the presidency. AT&T said it would not disclose the names of the other people and firms hired.

Mr. Quinn was promoted to replace AT&T’s longtime head of lobbying, James Cicconi, just before the company announced its merger with Time Warner. Mr. Quinn, who began working at AT&T in the 1980s, is well connected in the political circles of both parties. But he, and the rest of the company, was surprised by the election results and had few connections to Mr. Trump’s circles.

AT&T’s vast lobbying team, which includes more than 100 people, and public policy staff will now report to the company’s general counsel, David McAtee.

Mr. Quinn declined to comment.

Earlier this week, the company said that it had been contacted late last year about Mr. Cohen by the office of Robert S. Mueller III, the special counsel investigating Russian meddling in the 2016 election and other matters. AT&T said it had “cooperated fully” with the inquiries.

Novartis said this week that it had also spoken with Mr. Mueller’s team about the payments to Mr. Cohen. Novartis said that it had cooperated fully and considers its role in the matter closed.

Analysts said they did not expect the revelations about AT&T’s ties to Mr. Cohen to affect the government’s lawsuit to block the company’s merger with Time Warner.

AT&T and Time Warner had suggested ahead of the trial that the Justice Department’s decision to block a merger of two companies that don’t compete was influenced by presidential politics. Mr. Trump has been vocal in his disdain for CNN’s coverage of his administration. CNN is owned by Time Warner.

But Judge Richard J. Leon of United States District Court in Washington has been strict about keeping politics out of the case, which focuses on antitrust law and whether the deal would violate competition policy and harm consumers.

Judge Leon is expected to deliver an opinion on the case by June 12.

“These revelations come at a critical point in the trial, but they are very unlikely to have any meaningful impact on the judge’s ruling,” said Gene Kimmelman, a former senior official for the antitrust division of the Justice Department and the president of nonprofit Public Knowledge.
""")

In [67]:
correct, total = 0, 0
for s1, s2 in tqdm_notebook(pairwise(doc.sentences)):

    try:

        tokens1 = [str(t) for t in s1]
        tokens2 = [str(t) for t in s2]

        d1 = Document.from_text(' '.join(tokens1 + tokens2))
        d2 = Document.from_text(' '.join(tokens2 + tokens1))

        spans1 = model(d1)
        spans2 = model(d2)

        score1 = sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans1]).item()
        score2 = sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans2]).item()

        if score1 > score2:
            correct += 1

        total += 1

    except Exception as e:
        print(e)

HBox(children=(IntProgress(value=0, max=38), HTML(value='')))

In [68]:
correct / total

0.47368421052631576

In [53]:
colors = ('blue', 'green', 'red', 'orange', 'purple')

def viz_clusters(doc):
    
    clusters = model.predict(doc)
    spans = model(doc)
    
    tags = {
        i: tag
        for tag, c in enumerate(clusters)
        for i1, i2 in c
        for i in range(i1, i2+1)
    }
    
    tokens = []
    for i, token in enumerate(doc.tokens):
        tag = tags.get(i)
        color = colors[tag] if tag is not None else 'black'
        weight = 'bold' if tag is not None else 'regular'
        tokens.append(f'<span style="color: {color}; font-weight: {weight};">{token.text}</span>')
        
    display(HTML(' '.join(tokens)))
    print(sum([sum(F.softmax(s.sij, 0)[:-1]) for s in spans]).item())

In [54]:
viz_clusters(Document.from_text('David woke up. He went to the store.'))

0.9959083199501038


In [55]:
viz_clusters(Document.from_text('He went to the store. David woke up'))

0.5271236896514893


In [71]:
viz_clusters(Document.from_text('Shayne bought a car. Then he drove it to California.'))

1.8547711372375488


In [72]:
viz_clusters(Document.from_text('Then he drove it to California. Shayne bought a car.'))

0.6168746948242188


In [64]:
viz_clusters(Document.from_text('Mr. Quinn was promoted to replace AT&T’s longtime head of lobbying, James Cicconi, just before the company announced its merger with Time Warner. Mr. Quinn, who began working at AT&T in the 1980s, is well connected in the political circles of both parties. But he, and the rest of the company, was surprised by the election results and had few connections to Mr. Trump’s circles.'))

3.47507905960083


In [65]:
viz_clusters(Document.from_text(' But he, and the rest of the company, was surprised by the election results and had few connections to Mr. Trump’s circles. Mr. Quinn was promoted to replace AT&T’s longtime head of lobbying, James Cicconi, just before the company announced its merger with Time Warner. Mr. Quinn, who began working at AT&T in the 1980s, is well connected in the political circles of both parties.'))

2.9756550788879395
